In [ ]:
import sys
sys.path.append("../src/")
sys.path.append("src/")
from utils.utils import LocalModelArguments, LocalPLM

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "../models/TestLocal/checkpoint-19000",
    cuda_devices = "0",
    use_4bit_quantization = True,
    use_nested_quant = False,
    #attn_implementation = "eager"
)

In [ ]:
import gc
if "model" in locals():
    del model
gc.collect()

In [ ]:
model = LocalPLM(args)

In [ ]:
from datetime import datetime
local_time = datetime.now().strftime("%A, %B 2020, %-I:%M %p")

EXTRA_CONTEXT = """
You live away from Mum and Dad in an apartment in Hawthorn.
Your brother is steamingsheep (Nick) who lives in Melbourne.
You have a joke with Nick where you often talk about a dog named Grevald.
Your best friends are thisgreendingo (Liam), grumpy_koala (Callum), jam2see (James), and rastastherascal (Carlos) who all live in Melbourne.
Your enemy is AlzterBot (Fake Alex), a clone of yourself that you created using a fine-tuned Large Language Model.
""".strip().replace("\n", " ")

SYSTEM_PROMPT = f"""
You are alzter, a Discord user known for your art and programming skills.
Your real name is Alex, you live in Melbourne, Australia, and you study computer science at university.
{EXTRA_CONTEXT}
You often use internet slang and text emoticons in your messages. The time is {local_time}.
Do not repeat your previous messages when responding.""".strip().replace("\n", " ")

convo = [
    {"role":"system", "content":SYSTEM_PROMPT},
]

name = input("> What's your name?")
print(f"Starting conversation as {name}\n")

while True:
    user_input = input("> ")

    if user_input == "exit": break
    local_time = datetime.now().strftime("%A, %B 2020, %-I:%M %p")

    if user_input:
        user_input = user_input.replace("\\n", "\n")
        user_input = name + " (" + local_time + ")\n" + user_input
       # print("user: " + user_input)
        convo.append({"role":"user","content":user_input})
    
    response = model.generate(convo,temperature=1
                              , max_new_tokens=32)
    if not response.text: raise Exception("no response")

    message = response.text
    # Only capture the first 2 lines of Alzter's response.
    message = "\n".join(message.strip().split("\n")[:3])
    
    message = message.replace("\\n", "\n")
    message_readable = "\n".join(message.strip().split("\n")[1:])
    
    convo.append({"role":"assistant","content":message})
    print(message_readable)

In [ ]:
import pandas as pd
chat = pd.read_pickle("messages.pkl")

In [ ]:
import datetime
datetime.datetime.now(datetime.UTC)

In [ ]:
(chat["Date"].iloc[0] - datetime.datetime.now(datetime.UTC)).seconds // 60

In [ ]:
chat[ chat["Date"].apply(lambda x: datetime.datetime.now(datetime.UTC) - x < datetime.timedelta(minutes=10000)) ]